# How to use AnySDK to make a dynamic toolkit

AnySDK allows you to wrap any SDK client that exposes various CRUD operations (get, update, create, delete) and turn them into LLM-enabled toolkits without having to write a custom toolkit.

There are two core aspects to AnySDK:
1. AnySDKWrapper -- wraps the SDK you pass in
2. CrudControls -- allows you to control the access an agent has to Create, Read, Update, Delete verbs.

First, we'll create a client SDK. Let's use AWS's `boto3` library. Ensure you have your credentials. 

In [1]:
import boto3
client = boto3.client('s3')

Now, we can pass this client into AnySDKWrapper to create the toolkit. The callable functions within the SDK will become available as tools within the `anysdk.operations` object.

In [2]:
from langchain_community.utilities.anysdk import AnySdkWrapper

anysdk = AnySdkWrapper(
        client=client
    )

print(anysdk.operations)

[{'mode': 'create_bucket', 'name': 'Create Bucket', 'description': 

.. note::

  

  This action creates an Amazon S3 bucket. To create an Amazon S3 on Outposts bucket, see `CreateBucket <https://docs.aws.amazon.com/AmazonS3/latest/API/API_control_CreateBucket.html>`__.

  

 

Creates a new S3 bucket. To create a bucket, you must set up Amazon S3 and have a valid Amazon Web Services Access Key ID to authenticate requests. Anonymous requests are never allowed to create buckets. By creating the bucket, you become the bucket owner.

 

There are two types of buckets: general purpose buckets and directory buckets. For more information about these bucket types, see `Creating, configuring, and working with Amazon S3 buckets <https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-buckets-s3.html>`__ in the *Amazon S3 User Guide*.

 

.. note::

  

  
  * **General purpose buckets** - If you send your ``CreateBucket`` request to the ``s3.amazonaws.com`` global endpoint, the request 

Since `anysdk.operations` is a list of tools, we can pass that into an Agent upon initialization and use them.

In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.pretty_print()

================================ System Message ================================

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

==============

In [4]:
import getpass
import os

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

In [17]:
from langchain_anthropic import ChatAnthropic
from langchain.agents import Agent, create_structured_chat_agent, AgentExecutor

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

agent = create_structured_chat_agent(llm, anysdk.operations, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=anysdk.operations,
    verbose=True,
)


AttributeError: 'dict' object has no attribute 'args'

Now we can use this agent to perform functions for us using the configured AWS `boto3` client!

In [18]:
agent_executor.invoke(
    {
        "input": "How many S3 buckets do I have?"
    }
)

NameError: name 'agent_executor' is not defined